### Case study project exercise

Create a web API server that exposes process, memory and CPU usage information
of a cluster of nodes in a network.  

This project would consist of the following components:
  - An API server
    - This is a central component that should run on a host and provide services for fetching process, memory and CPU usage statistics of a given node in the network.
    - The server must also support gathering process, memory and CPU statistics via remote agents running on each node.
    - The server must persist all statistics on a database and allow fetching past details.
  - Agent
    - This is a python program that runs on each node to be monitored. 
    - This program must periodically fetch process, memory and CPU usage statistics of the current node and relay the same to the API server.
  - Client
    - This is a python program to test the server by fetching process, memory and CPU usage statistics using API call to the server

An example transcript of the client API call to the server:

```
    >>> requests.get("https://stats.testnet.net/nodes").json()
    [ "192.168.2.10", "192.168.2.11", "192.168.2.56" ]
    # Returns a list of host ip addresses being managed.

    >>> requests.get("https://stats.testnet.net/nodes/192.168.2.10").json()
    {"boottime": 4334434344.45,   # boot time in seconds since epoch
     "loadavg": [0.0, 0.4, 0.2],   # System load average
     "cpu_usage_percent": [12.4, 1.8, 8.7, 6.9], # Percentage of each CPU usage
     "memory_usage_percent": 32.0 # Memory usage in percentage
    }

    >>> requests.get("https://stats.testnet.net/nodes/192.168.2.10/cpu").json()
    { ... } # Fetch detailed CPU info for each CPU (frequency, usage, etc.)

    >>> requests.get("https://stats.testnet.net/nodes/192.168.2.10/memory").json()
    { ... } # Fetch details memory usage of the system (total memory, free memory, used memory, swap, etc...)

    >>> requests.get("https://stats.testnet.net/nodes/192.168.2.10/proc").json()
    [ 0, 1, 33, 4566, 7897, 34353, ... ] # Returns a list of PIDs

    >>> requests.get("https://stats.testnet.net/nodes/192.168.2.10/proc/0").json()
    {"pid": 0, "name": "System Idle process", "status": "running", ... }
    # Returns a detailed info on process resource usage (CPU, memory)
    
    >>> requests.get("https://stats.testnet.net/192.168.2.10/cpu", 
                     params={"from": "2022-03-01 13:45:00", 
                             "to": "2022-03-28 00:00:00"}).json()
    {"2022-03-01 13:45:00": {...}, "2022-03-01 13:47:00": {...}, ... }                         
    # Returns CPU stats report within the time period passed as parameters
    
# There is a lot of scope to improvise by supporting more usage statistics like
# disk I/O and network I/O

```

An example transcript of Agent communication to the API server:
```
   >>> requests.post("https://stats.testnet.net/nodes", 
                     json={"ip": "192.168.2.51", 
                           "hostname": "test-node.testnet.net", ...})
        # Add a new node to the managed node-list - run when agent is launched.

   >>> requests.put("https://stats.testnet.net/nodes/192.168.2.51", 
                    json={...})
        # Invoked periodically to update the CPU, memory, process statistics by
        # the agent (needs further improvisation for process statistics updates)

   >>> requests.delete("https://stats.testnet.net/nodes/192.168.2.51")
       # Invoked when the agent is being shutdown

```

The server, client and agent must be built as a python installable package that
can manage dependencies of third-party libraries. 